# Use Text-davinci-003 to evaluate generated answers in the GPTScore way

In [1]:
import os
import pandas as pd
import openai
from gpt_score.gpt3_score import gpt3score
from tqdm import tqdm

tqdm.pandas()

proxy = 'http://dell-1.star:7890' # 3090 docker
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

openai.api_key_path = ".openai-key2"

In [2]:
ANSWER_PROMPT = """请根据以下文本生成给定问题的答案，你的答案应该考虑与文本和问题的相关性、准确性，同时要考虑答案自身的有用性、深度、创造力和详细程度等因素。
[文本]: 
{context}

[问题]: 
{question}

[答案]:

"""


In [3]:
import re
from qa_generator import filter_qa

filename = "test/gpt_score/mannual-qa-raw.csv"
df_raw = pd.read_csv(filename)
question_list, answer_list, context_list = [], [], []
for idx, row in tqdm(df_raw.iterrows(), total=len(df_raw)):
    questions = re.split(r"\d+\.", row.questions)
    questions = [q.strip() for q in questions if q.strip()]
    answers = re.split(r"\d+\.", row.answers)
    answers = [a.strip() for a in answers if a.strip()]
    min_len = min(len(questions), len(answers))
    question_list.extend(questions[:min_len])
    answer_list.extend(answers[:min_len])
    context_list.extend([row.context] * min_len)
df_split = pd.DataFrame(
    {"question": question_list, "answer": answer_list, "context": context_list}
)
df_split.to_csv(filename.replace("-raw.csv", "-split.csv"), index=False)

[2023-07-10 22:41:50,298] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-10 22:41:55.427 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cuda
100%|██████████| 424/424 [00:00<00:00, 15777.01it/s]


In [4]:
filter_qa(filename.replace("-raw.csv", "-split.csv"))

before filter:  (1799, 3)
after length filter:  (1660, 3)
after question mark filter:  (1652, 3)
after period filter:  (1593, 3)
after key word filter:  (1557, 3)
after duplicate filter:  (1537, 3)


100%|██████████| 1537/1537 [00:01<00:00, 1348.88it/s]

after similarity filter:  (1083, 5)
after rouge filter:  (1068, 5)


'test/gpt_score/mannual-qa-split-filtered.csv'

In [8]:
def get_answer_gpt_score(row):
    input = ANSWER_PROMPT.format(context=row["context"], question=row["question"])
    output = row["answer"]
    score = gpt3score(input, output, gpt3model='davinci003')
    return score

In [12]:
df_filtered = pd.read_csv(filename.replace("-raw.csv", "-split-filtered.csv"))
df_filtered.rename(
    columns={"prompt": "question", "completion": "answer"}, inplace=True
)
df_filtered['gpt_score'] = df_filtered.progress_apply(get_answer_gpt_score, axis=1)

  0%|          | 0/1068 [00:00<?, ?it/s]

 40%|███▉      | 425/1068 [05:53<08:56,  1.20it/s]

InvalidRequestError
Prompt passed in:

请根据以下文本生成给定问题的答案，你的答案应该考虑与文本和问题的相关性、准确性，同时要考虑答案自身的有用性、深度、创造力和详细程度等因素。
[文本]: 
summary: 常用功能#手机营业厅网页版功能列表
content: 网页版手机营业厅wap.10010.com常用功能1、话费查询话费查询目前可提供当月话费、可用余额、历史帐单、信用额度、交费记录、SP代收费查询服务。用户可通过手机自助查询，随时掌握话费情况用户登录手机营业厅首页，输入手机号码+服务密码登录，找到“话费查询”菜单（iPhone手机为图标）即可查询话费情况1）当月话费用户通过当月话费查询可查到用户本月产生的话费总额，提供包括计费周期、当月话费、可用余额、信用额度、号码状态、话费明细表等信息2）可用余额用户通过可用余额可查到话费余额信息3）历史帐单用户通过历史帐单可查到过去5个月（不含当前月）的帐单信息，让您清晰地了解到历史消费明细4）信用额度用户通过信用额度可查到号码的信用额度值5）交费记录用户通过交费记录可查到最近6个月（含当前月）的交费情况，提供包括交费日期、交费方式、交费金额等信息5）SP代收费用户通过SP代收费可查到最近6个月（含当前月）您使用SP业务的费用情况2、交费充值1）充值卡充值用户已拥有全国一卡充充值卡，可通过手机营业厅无需用户登录即可为全国联通的手机、固话、小灵通、宽带用户充值充值流程第一步：登录手机营业厅首页wap.10010.com第二步：找到“交费充值”菜单，选择“充值卡为手机充值”或“充值卡为固话、小灵通、宽带充值第三步：输入充值号码第四步：输入已有的全国一卡充充值卡密码、四位验证码第五步：确认充值，充值完成3、3G业务办理1）3G套餐变更3G用户可通过“套餐变更”功能来变更基本套餐，新变更的套餐于次月1日起生效。用户每个月只能进行一次套餐变更操作用户登录手机营业厅首页，找到“业务办理”菜单，选择“3G套餐变更”，完成用户登录后，在可选套餐列表中选择新套餐，按“立即变更”按钮完成变更操作。（目前开通青岛、德州、枣庄、烟台、潍坊、泰安共6个地市2）手机上网流量短信提醒3G用户可通过“手机上网流量”开通、变更、取消手机上网流量提醒短信（即以短信方式告知您当月累计使用流量和收费流量情况，默认按周短信

 40%|███▉      | 426/1068 [05:53<07:27,  1.44it/s]

InvalidRequestError
Prompt passed in:

请根据以下文本生成给定问题的答案，你的答案应该考虑与文本和问题的相关性、准确性，同时要考虑答案自身的有用性、深度、创造力和详细程度等因素。
[文本]: 
summary: 常用功能#手机营业厅网页版功能列表
content: 网页版手机营业厅wap.10010.com常用功能1、话费查询话费查询目前可提供当月话费、可用余额、历史帐单、信用额度、交费记录、SP代收费查询服务。用户可通过手机自助查询，随时掌握话费情况用户登录手机营业厅首页，输入手机号码+服务密码登录，找到“话费查询”菜单（iPhone手机为图标）即可查询话费情况1）当月话费用户通过当月话费查询可查到用户本月产生的话费总额，提供包括计费周期、当月话费、可用余额、信用额度、号码状态、话费明细表等信息2）可用余额用户通过可用余额可查到话费余额信息3）历史帐单用户通过历史帐单可查到过去5个月（不含当前月）的帐单信息，让您清晰地了解到历史消费明细4）信用额度用户通过信用额度可查到号码的信用额度值5）交费记录用户通过交费记录可查到最近6个月（含当前月）的交费情况，提供包括交费日期、交费方式、交费金额等信息5）SP代收费用户通过SP代收费可查到最近6个月（含当前月）您使用SP业务的费用情况2、交费充值1）充值卡充值用户已拥有全国一卡充充值卡，可通过手机营业厅无需用户登录即可为全国联通的手机、固话、小灵通、宽带用户充值充值流程第一步：登录手机营业厅首页wap.10010.com第二步：找到“交费充值”菜单，选择“充值卡为手机充值”或“充值卡为固话、小灵通、宽带充值第三步：输入充值号码第四步：输入已有的全国一卡充充值卡密码、四位验证码第五步：确认充值，充值完成3、3G业务办理1）3G套餐变更3G用户可通过“套餐变更”功能来变更基本套餐，新变更的套餐于次月1日起生效。用户每个月只能进行一次套餐变更操作用户登录手机营业厅首页，找到“业务办理”菜单，选择“3G套餐变更”，完成用户登录后，在可选套餐列表中选择新套餐，按“立即变更”按钮完成变更操作。（目前开通青岛、德州、枣庄、烟台、潍坊、泰安共6个地市2）手机上网流量短信提醒3G用户可通过“手机上网流量”开通、变更、取消手机上网流量提醒短信（即以短信方式告知您当月累计使用流量和收费流量情况，默认按周短信

...

 71%|███████   | 757/1068 [10:25<01:57,  2.64it/s]

InvalidRequestError
Prompt passed in:

请根据以下文本生成给定问题的答案，你的答案应该考虑与文本和问题的相关性、准确性，同时要考虑答案自身的有用性、深度、创造力和详细程度等因素。
[文本]: 
summary: 3G重点业务#无线上网卡#常见问题
content: 1.办理无线上网卡业务需要什么手续答：用户在购买后付费产品时,需填写用户资料并办理入网手续，由系统激活后方可使用用户购买标准产品,无需任何手续，购卡后用户首次上网自动激活2.办理无线上网卡可否自行选号答：不提供自行选号服务3.无线上网卡可办理自备上网卡入网吗?我现在用的是联通130/156号码，可否直截了当购买上网卡使用答：上网卡以〝一卡一号〞产品包形式销售，不单独销售上网卡和USIM卡，不办理用户自备上网卡入网手续4.预付费用户超出锁定期后，要求复原使用如何办答：按重新入网办理，承诺使用原上网卡，但需换号。办理时用户须携带有效身份证件、原USIM卡和原用户卡〔大卡即卡托〕营业厅办理5.中国联通是否有上网本答。估量5月17日推出6.上网卡业务套餐自动升级模式是如何回事答：用户使用量超过所选套餐对应流量后，超出部分按照流量计费。当原套餐月费与超出部分使用费用之和达到更高一级套餐月费时，自动向更高一级套餐升级〔即收费及套餐内流量与更高一级套餐相一致〕；下月自动复原到用户选择的套餐档7.当原套餐月费与超出部分使用费用之和未达到更高一级套餐月费时，现套餐超出流量部分如何计费答：超出部分按照流量收费8.上网后如何查询使用的流量答：通过网上营业厅进行查询9.新入网的无线上网卡客户当月如何收费答：客户如于上半月选择套餐入网，默认配置最低档套餐，用户可更换套餐或选择标准资费。在选择套餐的情形下执行套餐自动升级客户如于下半月〔每月16日零时以后〕入网，客户所选择套餐月费和所含流量均减半，执行自动套餐升级。〔举例说明：如客户在下半月选择80元套餐入网，客户当月的套餐月费和所含流量均减半，为40元包500MB流量，当客户的使用费超过40元后，按照流量计费，当费用达到80元后，按80元包1GB流量的套餐资费执行，以后按照后续套餐档顺序升级。〕10.预付费用户通过一卡充及网上营业厅充值后，所充金额还能退还吗答：通过一卡充及网上营业厅充值后，所充金额概不退还11.预付费用户超出锁定期后，要求复原

 71%|███████   | 758/1068 [10:25<01:53,  2.74it/s]

InvalidRequestError
Prompt passed in:

请根据以下文本生成给定问题的答案，你的答案应该考虑与文本和问题的相关性、准确性，同时要考虑答案自身的有用性、深度、创造力和详细程度等因素。
[文本]: 
summary: 3G重点业务#无线上网卡#常见问题
content: 1.办理无线上网卡业务需要什么手续答：用户在购买后付费产品时,需填写用户资料并办理入网手续，由系统激活后方可使用用户购买标准产品,无需任何手续，购卡后用户首次上网自动激活2.办理无线上网卡可否自行选号答：不提供自行选号服务3.无线上网卡可办理自备上网卡入网吗?我现在用的是联通130/156号码，可否直截了当购买上网卡使用答：上网卡以〝一卡一号〞产品包形式销售，不单独销售上网卡和USIM卡，不办理用户自备上网卡入网手续4.预付费用户超出锁定期后，要求复原使用如何办答：按重新入网办理，承诺使用原上网卡，但需换号。办理时用户须携带有效身份证件、原USIM卡和原用户卡〔大卡即卡托〕营业厅办理5.中国联通是否有上网本答。估量5月17日推出6.上网卡业务套餐自动升级模式是如何回事答：用户使用量超过所选套餐对应流量后，超出部分按照流量计费。当原套餐月费与超出部分使用费用之和达到更高一级套餐月费时，自动向更高一级套餐升级〔即收费及套餐内流量与更高一级套餐相一致〕；下月自动复原到用户选择的套餐档7.当原套餐月费与超出部分使用费用之和未达到更高一级套餐月费时，现套餐超出流量部分如何计费答：超出部分按照流量收费8.上网后如何查询使用的流量答：通过网上营业厅进行查询9.新入网的无线上网卡客户当月如何收费答：客户如于上半月选择套餐入网，默认配置最低档套餐，用户可更换套餐或选择标准资费。在选择套餐的情形下执行套餐自动升级客户如于下半月〔每月16日零时以后〕入网，客户所选择套餐月费和所含流量均减半，执行自动套餐升级。〔举例说明：如客户在下半月选择80元套餐入网，客户当月的套餐月费和所含流量均减半，为40元包500MB流量，当客户的使用费超过40元后，按照流量计费，当费用达到80元后，按80元包1GB流量的套餐资费执行，以后按照后续套餐档顺序升级。〕10.预付费用户通过一卡充及网上营业厅充值后，所充金额还能退还吗答：通过一卡充及网上营业厅充值后，所充金额概不退还11.预付费用户超出锁定期后，要求复原

100%|██████████| 1068/1068 [15:09<00:00,  1.17it/s]


In [29]:
# normalize gpt_score
df_filtered['gpt_score'] = (df_filtered["gpt_score"] - df_filtered["gpt_score"].min()) \
    / (df_filtered["gpt_score"].max() - df_filtered["gpt_score"].min())
df_filtered.to_csv(filename.replace("-raw.csv", "-score.csv"), index=False)

In [30]:
# drop empty gpt_score column
df_valid = df_filtered.dropna(subset=['gpt_score'])

df_valid.sort_values(by="gpt_score", ascending=False) \
            [["question", "answer", "gpt_score", "similarity", "rouge"]] \
            .to_csv(filename.replace("-raw.csv", "-score-desend.csv"), index=False)

df_valid.sort_values(by="gpt_score", ascending=True) \
            [["question", "answer", "gpt_score", "similarity", "rouge"]] \
            .to_csv(filename.replace("-raw.csv", "-score-ascend.csv"), index=False)

In [31]:
df_valid.sort_values(by="gpt_score", ascending=True).head()

,question,answer,context,similarity,rouge,gpt_score
304,CDMA用户在香港漫游拨打回国内的拨号方式是什么？费用如何收取？,30*115%）/分钟。,summary: 法律常识#CDMA用户漫游香港拨打香港当地电话的拨号方式，费用如何收取？拨...,0.250407,0.000000,0.000000
362,17911接通率为什么不是很高？,因为接通率不是很高。,summary: 法律常识#17911为何经常出现打不通的现象\ncontent: 答案要点...,0.649739,0.615385,0.255876
912,如果重新启动modem后线路灯依然闪烁，应该怎么做？,打10010号报障。,summary: 根据ADSL指示灯排障步骤#线路灯：使用ADSL时，用户端的Moden要与...,0.304153,0.000000,0.318788
356,手机拨打澳门长途电话需要支付哪些费用？,2元/6秒的长话费。,summary: 法律常识#手机拨打澳门长途如何收费？是否有优惠时段\ncontent: 答...,0.460188,0.000000,0.351535
330,WAP业务的通信速度是多少？,WAP业务的端口号是9201。,summary: 法律常识#请写出WAP业务的通用设置\ncontent: 答案要点：接入号...,0.714699,0.533333,0.378367
